In [9]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
import warnings

from statsmodels.nonparametric.smoothers_lowess import lowess

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
save_path = Path('/Users/alina/Desktop/MIT/code/data/output/figures/mediator_regression_14months')

In [11]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived = '/Users/alina/Desktop/MIT/code/data/output/derived_data'

In [12]:
snap = pd.read_csv(Path(data_derived, 'snap_14_mediators.csv'))
ssrs = pd.read_csv(Path(data_derived, 'ssrs_14_mediators.csv'))
masc = pd.read_csv(Path(data_derived, 'masc_14_mediators.csv'))
pc = pd.read_csv(Path(data_derived, 'pc_14_mediators.csv'))
qsts =['snap', 'ssrs', 'masc', 'pc']
data_dict = dict(zip(['snap', 'ssrs', 'masc', 'pc'], [snap, ssrs, masc, pc]))

In [26]:
#outcome variablles 
snap_vars = ['snainatx', 'snahix', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
#wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes = np.concatenate([snap_vars, ssrs_vars, masc_vars, pc_vars])

outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars, 'masc': masc_vars, 'pc': pc_vars}#, 'wechsler': wechsler_vars}

In [13]:
for qst in data_dict.keys():
    print((qst))
    print((data_dict[qst]['days_baseline'].value_counts() < 0).sum())
    print('L', (data_dict[qst][ (data_dict[qst]['trtname'] == 'L') &(data_dict[qst]['days_baseline'] >=0)].shape))
    print('M', (data_dict[qst][ (data_dict[qst]['trtname'] == 'M') &(data_dict[qst]['days_baseline'] >=0)].shape))
    print('P', (data_dict[qst][ (data_dict[qst]['trtname'] == 'P') &(data_dict[qst]['days_baseline'] >=0)].shape))
    print('C', (data_dict[qst][ (data_dict[qst]['trtname'] == 'C') &(data_dict[qst]['days_baseline'] >=0)].shape))
    print('A', (data_dict[qst][ (data_dict[qst]['trtname'] == 'A') &(data_dict[qst]['days_baseline'] >=0)].shape))
    

snap
0
L (0, 17)
M (985, 17)
P (1118, 17)
C (1137, 17)
A (1033, 17)
ssrs
0
L (0, 16)
M (996, 16)
P (1114, 16)
C (1135, 16)
A (1040, 16)
masc
0
L (0, 14)
M (336, 14)
P (382, 14)
C (372, 14)
A (340, 14)
pc
0
L (0, 15)
M (1218, 15)
P (1359, 15)
C (1382, 15)
A (1257, 15)


In [42]:
qsts

['snap', 'ssrs', 'masc', 'pc']

In [14]:
ids_A = []
for qst in qsts :
    ids_A = data_dict[qst][ data_dict[qst]['trtname'] == 'A'].src_subject_id.unique()
    print(qst, 'M', ids_A.any() in data_dict[qst][ data_dict[qst]['trtname'] == 'M'])
    print(qst, "C", ids_A.any() in data_dict[qst][ data_dict[qst]['trtname'] == 'C'])
    print(qst, "P", ids_A.any() in data_dict[qst][ data_dict[qst]['trtname'] == 'P'])


snap M False
snap C False
snap P False
ssrs M False
ssrs C False
ssrs P False
masc M False
masc C False
masc P False
pc M False
pc C False
pc P False


In [74]:
outcomes_dict

written_out_dict = {'snainatx' : "natten", 'snahix': "yperact", 'snaoddx': "gress", 
                     'sspintx': "ntern", 'ssptossx':"SSRS",
                     'masc_masctotalt': 'MASC',
                     'pcrcpax': "ower", 'pcrcprx': "lose"}
written_out_dict

{'snainatx': 'natten',
 'snahix': 'yperact',
 'snaoddx': 'gress',
 'sspintx': 'ntern',
 'ssptossx': 'SSRS',
 'masc_masctotalt': 'MASC',
 'pcrcpax': 'ower',
 'pcrcprx': 'lose'}

In [45]:
data_dict[qst]["trtname"].unique()

array(['C', 'M', 'A', 'P'], dtype=object)

In [36]:
means_dict = {}

for qst in qsts:
    outcomes = outcomes_dict[qst]
    means_trt ={}
    for trt in data_dict[qst]["trtname"].unique():
        data = data_dict[qst][(data_dict[qst]["days_baseline"]== 0) & (data_dict[qst]["trtname"]== trt)  ]
        mean = [{out: data[out].mean(0)} for out in outcomes]
        means_trt[trt] = mean
    means_dict[qst] = means_trt
    

excellent. now, given that the data represents mean, std and number of subjects for each of those measures at baseline, and that the code to extract the means is the following :for qst in qsts:
    outcomes = outcomes_dict[qst]
    means_trt ={}
    for trt in data_dict[qst]["trtname"].unique():
        data = data_dict[qst][(data_dict[qst]["days_baseline"]== 0) & (data_dict[qst]["trtname"]== trt)  ]

write a code snippet that instead of putting the calculated means ina dictionnary, polulate the appropriate cells in the dataframe. For you information , qsts  = ['snap', 'ssrs', 'masc', 'pc'] respresent the different questionnairesnames, which are also the keys to acces the data in data_dict. days_baseline = 0 indicated that we extract only datapoints at baseline, and we loop overdata_dict[qst]["trtname"].unique() = ['C', 'M', 'A', 'P'], which are the treatment names, where C corresponds to "combined treatment", "M" corresponds to medication management, "P" corresponds to behavioral treatment, and "A" corresponds to community caere 

In [37]:
means_dict

{'snap': {'C': [{'snainatx': 1.977420382165605},
   {'snahix': 1.8511146496815287},
   {'snaoddx': 1.3163375796178343}],
  'M': [{'snainatx': 2.0968120805369126},
   {'snahix': 1.9044295302013423},
   {'snaoddx': 1.4511073825503353}],
  'A': [{'snainatx': 2.061917808219178},
   {'snahix': 1.8382534246575342},
   {'snaoddx': 1.3517123287671233}],
  'P': [{'snainatx': 2.0377848101265825},
   {'snahix': 1.8571518987341769},
   {'snaoddx': 1.3417721518987342}]},
 'ssrs': {'C': [{'sspintx': 0.881823708206687},
   {'ssptossx': 0.9917021276595744}],
  'M': [{'sspintx': 0.8959405940594061}, {'ssptossx': 0.9519141914191419}],
  'A': [{'sspintx': 0.874418604651163}, {'ssptossx': 0.9778737541528241}],
  'P': [{'sspintx': 0.8985579937304075}, {'ssptossx': 0.9500313479623824}]},
 'masc': {'C': [{'masc_masctotalt': 109.76190476190476}],
  'M': [{'masc_masctotalt': 105.47457627118644}],
  'A': [{'masc_masctotalt': 110.0}],
  'P': [{'masc_masctotalt': 103.93650793650794}]},
 'pc': {'C': [{'pcrcpax': 2

In [49]:
df_outcomes = pd.read_csv(Path(data_derived, "outcome_means_b_14.csv"))

In [50]:
df_outcomes

,Unnamed: 0,Outcome Domain,Measure and Rater,Combined Treatment - Baseline (SD) [No. of Subjects],Combined Treatment - 14 mo (SD) [No. of Subjects],Medication Management - Baseline (SD) [No. of Subjects],Medication Management - 14 mo (SD) [No. of Subjects],Behavioral Treatment - Baseline (SD) [No. of Subjects],Behavioral Treatment - 14 mo (SD) [No. of Subjects],Assessment and Referral - Baseline (SD) [No. of Subjects],Assessment and Referral - 14 mo (SD) [No. of Subjects]
0,0,ADHD symptoms,Inattention - Teacher,2.16 (0.67) [137],1.12 (0.75) [134],2.27 (0.61) [135],1.11 (0.77) [120],2.28 (0.64) [136],1.47 (0.81) [119],2.19 (0.69) [135],1.48 (0.82) [128]
1,1,ADHD symptoms,Inattention - Parent,2.07 (0.61) [140],1.02 (0.66) [133],2.03 (0.64) [140],1.12 (0.70) [121],1.99 (0.63) [139],1.40 (0.68) [129],2.05 (0.65) [142],1.49 (0.67) [130]
2,2,ADHD symptoms,Hyperactive/impulsive - Teacher,1.89 (0.77) [137],0.75 (0.71) [134],2.08 (0.71) [135],0.82 (0.69) [120],2.05 (0.75) [136],1.10 (0.77) [119],1.93 (0.81) [135],1.25 (0.84) [128]
3,3,ADHD symptoms,Hyperactive/impulsive - Parent,1.91 (0.69) [140],1.85 (0.63) [133],1.89 (0.62) [140],0.91 (0.65) [121],1.89 (0.64) [140],1.24 (0.72) [129],1.95 (0.67) [142],1.35 (0.72) [130]
4,4,ADHD symptoms,Hyperactive/impulsive - Classroom observer,0.33 (0.22) [122],0.21 (0.20) [114],0.31 (0.21) [119],0.16 (0.15) [110],0.37 (0.26) [120],0.29 (0.26) [107],0.38 (0.27) [118],0.18 (0.15) [109]
5,5,Aggression-ODD,ODD aggression - Teacher,1.29 (0.91) [137],0.61 (0.68) [134],1.39 (0.92) [120],0.65 (0.68) [120],1.43 (0.86) [136],0.97 (0.80) [119],1.35 (0.88) [135],1.00 (0.84) [128]
6,6,Aggression-ODD,ODD aggression - Parent,1.39 (0.71) [140],0.76 (0.64) [133],1.45 (0.80) [139],0.94 (0.74) [121],1.37 (0.70) [140],1.05 (0.74) [129],1.49 (0.70) [142],1.11 (0.67) [130]
7,7,Aggression-ODD,ODD aggression - Classroom observer,0.018 (0.038) [122],0.007 (0.015) [114],0.014 (0.025) [119],0.004 (0.011) [108],0.020 (0.046) [120],0.010 (0.018) [107],0.019 (0.026) [118],0.006 (0.014) [109]
8,8,Internalizing symptoms,SSRS internalizing symptoms - Teacher,0.73 (0.51) [113],0.68 (0.44) [108],0.79 (0.47) [117],0.63 (0.47) [99],0.82 (0.45) [115],0.58 (0.40) [102],0.78 (0.44) [115],0.69 (0.44) [105]
9,9,Internalizing symptoms,SSRS internalizing symptoms - Parent,0.98 (0.37) [138],0.67 (0.37) [127],0.97 (0.37) [137],0.67 (0.39) [120],0.93 (0.43) [133],0.77 (0.40) [131],0.97 (0.35) [137],0.82 (0.43) [125]


In [52]:
df_baseline_values = df_outcomes[
    [
        'Outcome Domain', 
        'Measure and Rater',
        'Combined Treatment - Baseline (SD) [No. of Subjects]',
        'Medication Management - Baseline (SD) [No. of Subjects]',
        'Behavioral Treatment - Baseline (SD) [No. of Subjects]',
        'Assessment and Referral - Baseline (SD) [No. of Subjects]'
    ]
]

# Display the new DataFrame
print(type(df_baseline_values))

<class 'pandas.core.frame.DataFrame'>


In [82]:
df_14_values = df_outcomes[
    [
        'Outcome Domain', 
        'Measure and Rater',
        'Combined Treatment - 14 mo (SD) [No. of Subjects]',
        'Medication Management - 14 mo (SD) [No. of Subjects]',
        'Behavioral Treatment - 14 mo (SD) [No. of Subjects]',
        'Assessment and Referral - 14 mo (SD) [No. of Subjects]'
    ]
]

# Display the new DataFrame
df_14_values

,Outcome Domain,Measure and Rater,Combined Treatment - 14 mo (SD) [No. of Subjects],Medication Management - 14 mo (SD) [No. of Subjects],Behavioral Treatment - 14 mo (SD) [No. of Subjects],Assessment and Referral - 14 mo (SD) [No. of Subjects]
0,ADHD symptoms,Inattention - Teacher,1.12 (0.75) [134],1.11 (0.77) [120],1.47 (0.81) [119],1.48 (0.82) [128]
1,ADHD symptoms,Inattention - Parent,1.02 (0.66) [133],1.12 (0.70) [121],1.40 (0.68) [129],1.49 (0.67) [130]
2,ADHD symptoms,Hyperactive/impulsive - Teacher,0.75 (0.71) [134],0.82 (0.69) [120],1.10 (0.77) [119],1.25 (0.84) [128]
3,ADHD symptoms,Hyperactive/impulsive - Parent,1.85 (0.63) [133],0.91 (0.65) [121],1.24 (0.72) [129],1.35 (0.72) [130]
4,ADHD symptoms,Hyperactive/impulsive - Classroom observer,0.21 (0.20) [114],0.16 (0.15) [110],0.29 (0.26) [107],0.18 (0.15) [109]
5,Aggression-ODD,ODD aggression - Teacher,0.61 (0.68) [134],0.65 (0.68) [120],0.97 (0.80) [119],1.00 (0.84) [128]
6,Aggression-ODD,ODD aggression - Parent,0.76 (0.64) [133],0.94 (0.74) [121],1.05 (0.74) [129],1.11 (0.67) [130]
7,Aggression-ODD,ODD aggression - Classroom observer,0.007 (0.015) [114],0.004 (0.011) [108],0.010 (0.018) [107],0.006 (0.014) [109]
8,Internalizing symptoms,SSRS internalizing symptoms - Teacher,0.68 (0.44) [108],0.63 (0.47) [99],0.58 (0.40) [102],0.69 (0.44) [105]
9,Internalizing symptoms,SSRS internalizing symptoms - Parent,0.67 (0.37) [127],0.67 (0.39) [120],0.77 (0.40) [131],0.82 (0.43) [125]


In [63]:
outcomes_dict

{'snap': ['snainatx', 'snahix', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx']}

In [85]:
time = "14"

# add sepration rater 


for qst in qsts:
    outcomes = outcomes_dict[qst]  # Outcomes of interest for the current questionnaire

    # Loop over each treatment
    for trt in data_dict[qst]["trtname"].unique():
        # Filter the data for baseline (days_baseline = 0) and the current treatment
        if time == "b":
            conditions = (data_dict[qst]["days_baseline"] == 0) & (data_dict[qst]["trtname"] == trt)
            df = df_baseline_values
        elif time == "14":
            conditions = (data_dict[qst]["days_baseline"] > 150 ) &(data_dict[qst]["days_baseline"] < 350)  & (data_dict[qst]["trtname"] == trt)
            df = df_14_values
            
        data = data_dict[qst][conditions]
        
        # Calculate the mean, std, and number of subjects for each outcome
        for outcome in outcomes:
            print(outcome)
            mean_value = data[outcome].mean(0)
            std_value = data[outcome].std()
            Nsubjects = data[outcome].count()
            print("mean", mean_value, "std", std_value, Nsubjects)
            # Format the string as "mean (std) [Nsubjects]"
            formatted_value = f"{mean_value:.2f} ({std_value:.2f}) [{Nsubjects}]"
            print("F", formatted_value)

            # Determine the corresponding column name for the replicated data based on the treatment
            if trt == 'C':
                column_name = 'Replicated Combined Treatment - Baseline (SD) [No. of Subjects]'
            elif trt == 'M':
                column_name = 'Replicated Medication Management - Baseline (SD) [No. of Subjects]'
            elif trt == 'P':
                column_name = 'Replicated Behavioral Treatment - Baseline (SD) [No. of Subjects]'
            elif trt == 'A':
                column_name = 'Replicated Assessment and Referral - Baseline (SD) [No. of Subjects]'
            
            # Add the new column if it doesn't exist in the DataFrame
            if column_name not in df.columns:
                df[column_name] = ""

            # Locate the appropriate row in the DataFrame by matching the outcome and qst
            row_index = df[
                
                (df['Measure and Rater'].str.contains(written_out_dict[outcome]))
            ].index
            
            # Update the DataFrame with the calculated formatted string in the new column
            df.loc[row_index, column_name] = formatted_value

snainatx
mean 1.0755873015873014 std 0.6447706508479777 315
F 1.08 (0.64) [315]
snahix
mean 0.9048571428571428 std 0.6497757294906626 315
F 0.90 (0.65) [315]
snaoddx
mean 0.7263809523809523 std 0.634405469926746 315
F 0.73 (0.63) [315]
snainatx
mean 1.0980851063829786 std 0.672942669287286 282
F 1.10 (0.67) [282]
snahix
mean 0.9053191489361703 std 0.6190764021763853 282
F 0.91 (0.62) [282]
snaoddx
mean 0.819822695035461 std 0.7045901641464547 282
F 0.82 (0.70) [282]
snainatx
mean 1.5792358803986712 std 0.7471798628899909 301
F 1.58 (0.75) [301]
snahix
mean 1.3182724252491695 std 0.7006737273537272 301
F 1.32 (0.70) [301]
snaoddx
mean 1.092159468438538 std 0.7617118797933922 301
F 1.09 (0.76) [301]
snainatx
mean 1.5535855263157896 std 0.7313033524613259 304
F 1.55 (0.73) [304]
snahix
mean 1.2425328947368421 std 0.7142661069388816 304
F 1.24 (0.71) [304]
snaoddx
mean 1.0358223684210526 std 0.7531346909632605 304
F 1.04 (0.75) [304]
sspintx
mean 0.7169867549668875 std 0.40799386821584355 

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_26984/2828204871.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = ""
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_26984/2828204871.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = ""
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_26984/2828204871.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [66]:
written_out_dict

{'snainatx': 'inatten',
 'snahix': 'hyperact',
 'snaoddx': 'agress',
 'sspintx': 'intern',
 'ssptossx': 'social',
 'masc_masctotalt': 'masc',
 'pcrcpax': 'power',
 'pcrcprx': 'close'}

In [84]:
df_baseline_values

,Outcome Domain,Measure and Rater,Combined Treatment - Baseline (SD) [No. of Subjects],Medication Management - Baseline (SD) [No. of Subjects],Behavioral Treatment - Baseline (SD) [No. of Subjects],Assessment and Referral - Baseline (SD) [No. of Subjects],Replicated Combined Treatment - Baseline (SD) [No. of Subjects],Replicated Medication Management - Baseline (SD) [No. of Subjects],Replicated Assessment and Referral - Baseline (SD) [No. of Subjects],Replicated Behavioral Treatment - Baseline (SD) [No. of Subjects]
0,ADHD symptoms,Inattention - Teacher,2.16 (0.67) [137],2.27 (0.61) [135],2.28 (0.64) [136],2.19 (0.69) [135],1.98 (0.67) [314],2.10 (0.62) [298],2.06 (0.68) [292],2.04 (0.68) [316]
1,ADHD symptoms,Inattention - Parent,2.07 (0.61) [140],2.03 (0.64) [140],1.99 (0.63) [139],2.05 (0.65) [142],1.98 (0.67) [314],2.10 (0.62) [298],2.06 (0.68) [292],2.04 (0.68) [316]
2,ADHD symptoms,Hyperactive/impulsive - Teacher,1.89 (0.77) [137],2.08 (0.71) [135],2.05 (0.75) [136],1.93 (0.81) [135],1.85 (0.70) [314],1.90 (0.67) [298],1.84 (0.71) [292],1.86 (0.71) [316]
3,ADHD symptoms,Hyperactive/impulsive - Parent,1.91 (0.69) [140],1.89 (0.62) [140],1.89 (0.64) [140],1.95 (0.67) [142],1.85 (0.70) [314],1.90 (0.67) [298],1.84 (0.71) [292],1.86 (0.71) [316]
4,ADHD symptoms,Hyperactive/impulsive - Classroom observer,0.33 (0.22) [122],0.31 (0.21) [119],0.37 (0.26) [120],0.38 (0.27) [118],1.85 (0.70) [314],1.90 (0.67) [298],1.84 (0.71) [292],1.86 (0.71) [316]
5,Aggression-ODD,ODD aggression - Teacher,1.29 (0.91) [137],1.39 (0.92) [120],1.43 (0.86) [136],1.35 (0.88) [135],1.32 (0.79) [314],1.45 (0.83) [298],1.35 (0.76) [292],1.34 (0.78) [316]
6,Aggression-ODD,ODD aggression - Parent,1.39 (0.71) [140],1.45 (0.80) [139],1.37 (0.70) [140],1.49 (0.70) [142],1.32 (0.79) [314],1.45 (0.83) [298],1.35 (0.76) [292],1.34 (0.78) [316]
7,Aggression-ODD,ODD aggression - Classroom observer,0.018 (0.038) [122],0.014 (0.025) [119],0.020 (0.046) [120],0.019 (0.026) [118],1.32 (0.79) [314],1.45 (0.83) [298],1.35 (0.76) [292],1.34 (0.78) [316]
8,Internalizing symptoms,SSRS internalizing symptoms - Teacher,0.73 (0.51) [113],0.79 (0.47) [117],0.82 (0.45) [115],0.78 (0.44) [115],0.99 (0.28) [329],0.95 (0.28) [303],0.98 (0.27) [301],0.95 (0.27) [319]
9,Internalizing symptoms,SSRS internalizing symptoms - Parent,0.98 (0.37) [138],0.97 (0.37) [137],0.93 (0.43) [133],0.97 (0.35) [137],0.99 (0.28) [329],0.95 (0.28) [303],0.98 (0.27) [301],0.95 (0.27) [319]


In [79]:
df_14_values = df_outcomes[
    [
        'Outcome Domain', 
        'Measure and Rater',
        'Combined Treatment - 14 mo (SD) [No. of Subjects]',
        'Medication Management - 14 mo (SD) [No. of Subjects]',
        'Behavioral Treatment - 14 mo (SD) [No. of Subjects]',
        'Assessment and Referral - 14 mo (SD) [No. of Subjects]'
    ]
]

# Display the new DataFrame
df_14_values

,Outcome Domain,Measure and Rater,Combined Treatment - 14 mo (SD) [No. of Subjects],Medication Management - 14 mo (SD) [No. of Subjects],Behavioral Treatment - 14 mo (SD) [No. of Subjects],Assessment and Referral - 14 mo (SD) [No. of Subjects]
0,ADHD symptoms,Inattention - Teacher,1.12 (0.75) [134],1.11 (0.77) [120],1.47 (0.81) [119],1.48 (0.82) [128]
1,ADHD symptoms,Inattention - Parent,1.02 (0.66) [133],1.12 (0.70) [121],1.40 (0.68) [129],1.49 (0.67) [130]
2,ADHD symptoms,Hyperactive/impulsive - Teacher,0.75 (0.71) [134],0.82 (0.69) [120],1.10 (0.77) [119],1.25 (0.84) [128]
3,ADHD symptoms,Hyperactive/impulsive - Parent,1.85 (0.63) [133],0.91 (0.65) [121],1.24 (0.72) [129],1.35 (0.72) [130]
4,ADHD symptoms,Hyperactive/impulsive - Classroom observer,0.21 (0.20) [114],0.16 (0.15) [110],0.29 (0.26) [107],0.18 (0.15) [109]
5,Aggression-ODD,ODD aggression - Teacher,0.61 (0.68) [134],0.65 (0.68) [120],0.97 (0.80) [119],1.00 (0.84) [128]
6,Aggression-ODD,ODD aggression - Parent,0.76 (0.64) [133],0.94 (0.74) [121],1.05 (0.74) [129],1.11 (0.67) [130]
7,Aggression-ODD,ODD aggression - Classroom observer,0.007 (0.015) [114],0.004 (0.011) [108],0.010 (0.018) [107],0.006 (0.014) [109]
8,Internalizing symptoms,SSRS internalizing symptoms - Teacher,0.68 (0.44) [108],0.63 (0.47) [99],0.58 (0.40) [102],0.69 (0.44) [105]
9,Internalizing symptoms,SSRS internalizing symptoms - Parent,0.67 (0.37) [127],0.67 (0.39) [120],0.77 (0.40) [131],0.82 (0.43) [125]


In [86]:
df_14_values

,Outcome Domain,Measure and Rater,Combined Treatment - 14 mo (SD) [No. of Subjects],Medication Management - 14 mo (SD) [No. of Subjects],Behavioral Treatment - 14 mo (SD) [No. of Subjects],Assessment and Referral - 14 mo (SD) [No. of Subjects],Replicated Combined Treatment - Baseline (SD) [No. of Subjects],Replicated Medication Management - Baseline (SD) [No. of Subjects],Replicated Assessment and Referral - Baseline (SD) [No. of Subjects],Replicated Behavioral Treatment - Baseline (SD) [No. of Subjects]
0,ADHD symptoms,Inattention - Teacher,1.12 (0.75) [134],1.11 (0.77) [120],1.47 (0.81) [119],1.48 (0.82) [128],1.08 (0.64) [315],1.10 (0.67) [282],1.58 (0.75) [301],1.55 (0.73) [304]
1,ADHD symptoms,Inattention - Parent,1.02 (0.66) [133],1.12 (0.70) [121],1.40 (0.68) [129],1.49 (0.67) [130],1.08 (0.64) [315],1.10 (0.67) [282],1.58 (0.75) [301],1.55 (0.73) [304]
2,ADHD symptoms,Hyperactive/impulsive - Teacher,0.75 (0.71) [134],0.82 (0.69) [120],1.10 (0.77) [119],1.25 (0.84) [128],0.90 (0.65) [315],0.91 (0.62) [282],1.32 (0.70) [301],1.24 (0.71) [304]
3,ADHD symptoms,Hyperactive/impulsive - Parent,1.85 (0.63) [133],0.91 (0.65) [121],1.24 (0.72) [129],1.35 (0.72) [130],0.90 (0.65) [315],0.91 (0.62) [282],1.32 (0.70) [301],1.24 (0.71) [304]
4,ADHD symptoms,Hyperactive/impulsive - Classroom observer,0.21 (0.20) [114],0.16 (0.15) [110],0.29 (0.26) [107],0.18 (0.15) [109],0.90 (0.65) [315],0.91 (0.62) [282],1.32 (0.70) [301],1.24 (0.71) [304]
5,Aggression-ODD,ODD aggression - Teacher,0.61 (0.68) [134],0.65 (0.68) [120],0.97 (0.80) [119],1.00 (0.84) [128],0.73 (0.63) [315],0.82 (0.70) [282],1.09 (0.76) [301],1.04 (0.75) [304]
6,Aggression-ODD,ODD aggression - Parent,0.76 (0.64) [133],0.94 (0.74) [121],1.05 (0.74) [129],1.11 (0.67) [130],0.73 (0.63) [315],0.82 (0.70) [282],1.09 (0.76) [301],1.04 (0.75) [304]
7,Aggression-ODD,ODD aggression - Classroom observer,0.007 (0.015) [114],0.004 (0.011) [108],0.010 (0.018) [107],0.006 (0.014) [109],0.73 (0.63) [315],0.82 (0.70) [282],1.09 (0.76) [301],1.04 (0.75) [304]
8,Internalizing symptoms,SSRS internalizing symptoms - Teacher,0.68 (0.44) [108],0.63 (0.47) [99],0.58 (0.40) [102],0.69 (0.44) [105],1.19 (0.25) [302],1.14 (0.28) [281],1.08 (0.28) [293],1.10 (0.27) [302]
9,Internalizing symptoms,SSRS internalizing symptoms - Parent,0.67 (0.37) [127],0.67 (0.39) [120],0.77 (0.40) [131],0.82 (0.43) [125],1.19 (0.25) [302],1.14 (0.28) [281],1.08 (0.28) [293],1.10 (0.27) [302]
